In [6]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import glob
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship


In [13]:

#creo el entorno
engine = create_engine('sqlite:///deteccion_plagio_tp2.db')

#creo la base
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

# Actualiza las tablas en la base de datos
Base.metadata.create_all(engine)

#Defino las tablas

class Documentos(Base):
    __tablename__ = 'documentos'
    id = Column(Integer, primary_key=True)
    nombre_archivo = Column(String)
    texto = Column(String)
    
class Segmentos(Base):
    __tablename__ = 'segmentos'
    id_segmento = Column(Integer, primary_key=True)
    id_documento = Column(Integer, ForeignKey('documentos.id'))
    segmento_texto = Column(String)
    segmento_limpio = Column(String)
    init_s = Column(String)
    texto = Column(String)
    
    documento = relationship("Documentos", backref="segmentos")

<ipython-input-13-f61fc0281007>:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [19]:
# Función para guardar el contenido de un archivo en la base de datos
def guardar_documento(ruta_archivo):
    nombre_archivo = os.path.basename(ruta_archivo)
    with open(ruta_archivo, 'r', encoding= "utf-8-sig") as archivo:
        contenido = archivo.read()

    nuevo_documento = Documentos(nombre_archivo=nombre_archivo, texto=contenido)
    session.add(nuevo_documento)

    try:
        session.commit()
        print(f"Documento '{nombre_archivo}' guardado correctamente.")
    except IntegrityError:
        session.rollback()
        print(f"Error al guardar el documento '{nombre_archivo}'.")



In [20]:
def countFiles(path, extension="*.txt"): 
    
    # Construir el patrón de búsqueda
    patron = os.path.join(path, extension)
    
    # Obtener la lista de archivos que coinciden con el patrón
    archivos = glob.glob(patron)
    
    return len(archivos)   

In [21]:
def load_corpus_documents(carpeta):

    # Recorre los archivos de la carpeta
    for archivo in os.listdir(carpeta):
        ruta_archivo = os.path.join(carpeta, archivo)
        if os.path.isfile(ruta_archivo) and archivo.endswith('.txt'):
            guardar_documento(ruta_archivo)

In [22]:
def main():

    
    #Defino la carpeta donde esta el corpus y guardo los archivos en la base
    path = os.path.join(os.getcwd(), "corpus")
    load_corpus_documents(path)

In [23]:
main()

Documento 'juan.txt' guardado correctamente.
Documento 'pepe.txt' guardado correctamente.
